In [ ]:
%%time

import polars as pl


# change the configsetting, to see the full tables
pl.Config.set_tbl_rows(100)
pl.Config(fmt_str_lengths=550)


# load one "compound" file (~4.2 Gb)
path = "../../../MINE-Database/data/output/metacyc_generalized/20240601_lotus_generalized_n45/compounds_1_generalized_230106_frozen_metadata_for_MINES_split_*1.csv*.parquet"

# load in with polars and filter out only the predicted -> Lazydataframe
df_predicted = (
    pl.scan_parquet(path).filter(pl.col("Type") == "Predicted")
    )

df = df_predicted.collect()

In [2]:
df.shape

df

(120378471, 6)

In [10]:
import sys

def check_element_in_list(element, lst, verbose=False):
    try:
        if element not in lst:
            if verbose:
                print(f"Element {element} not found in the list. \nPlease choose from the following columns:\n{lst}")
            sys.exit(1)  # Exit the script with a non-zero status
        else:
            if verbose:
                print(f"Element {element} is in the list.")
    except Exception as e:
        print(f"An error occurred: {e}")
        sys.exit(1)


def duplicates_amount(lazy_df, column_to_search=""):
    """
    """
    all_columns = df_predicted.columns

    # Specify the column you want to check for duplicates
    if not column_to_search:
        column_to_search = all_columns
    else:
        check_element_in_list(column_to_search, all_columns)

    # Find duplicates and collect them (for lazy dataframes)
    df_duplicates = df_predicted.group_by(column_to_search).agg(pl.len()).filter(pl.col("len") > 1)
    df_duplicates_collected = df_duplicates.collect()

    return df_duplicates_collected

In [11]:
%%time 

# choose a column - ['ID', 'Type', 'Generation', 'Formula', 'InChIKey', 'SMILES']
all_columns = df_predicted.columns
column_name = "InChIKey"

# search for duplicates (gives back a dataframe - collected)
df_duplicates_collected = duplicates_amount(df_predicted, column_to_search = column_name)

# Print the duplicates
print(f'duplicates in {column_name}\n\n{df_duplicates_collected}\n---\n')

duplicates in InChIKey

shape: (5_856_617, 2)
┌─────────────────────────────┬─────┐
│ InChIKey                    ┆ len │
│ ---                         ┆ --- │
│ str                         ┆ u32 │
╞═════════════════════════════╪═════╡
│ GBPWROKTDBEDRP-UHFFFAOYSA-N ┆ 2   │
│ ISYWXXNLLAPDBH-UHFFFAOYSA-N ┆ 2   │
│ AGFIXWAZWVWOFN-UHFFFAOYSA-N ┆ 2   │
│ IEGVJHQWKTYPHX-UHFFFAOYSA-N ┆ 2   │
│ PXIJFCQHORCHIC-UHFFFAOYSA-N ┆ 2   │
│ …                           ┆ …   │
│ AOFBKZGFAAPNCV-UHFFFAOYSA-N ┆ 2   │
│ BTNYQXQCUORBMC-UHFFFAOYSA-N ┆ 3   │
│ QKRLRASFOJSRIU-UHFFFAOYSA-N ┆ 2   │
│ OMPGVGWQTUDHJF-UHFFFAOYSA-N ┆ 2   │
│ RQAZWGMMQKFKDT-UHFFFAOYSA-N ┆ 2   │
└─────────────────────────────┴─────┘
---

CPU times: user 55.9 s, sys: 9.07 s, total: 1min 4s
Wall time: 37.9 s


In [16]:
print(f'duplications: {df_duplicates_collected["len"].unique()}')

df_duplicates_collected.filter(pl.col("len")==5)

duplications: shape: (4,)
Series: 'len' [u32]
[
	2
	3
	4
	5
]


InChIKey,len
str,u32
"""ZAHBPYBIXMGFFK…",5
"""ILYVLUHOJHGJKX…",5
"""IOTDQPHYHNCXAW…",5
"""LDXGPSPFDXHTIM…",5
"""AKQFFUOKLHMDMS…",5
…,…
"""PATHBLPNVPVFOE…",5
"""JJPAAWDFFPTLNU…",5
"""VKCHEYIDBYCQPR…",5


In [46]:
# load one "compound" file (~4.2 Gb)
path = "../../../MINE-Database/data/output/metacyc_generalized/20240601_lotus_generalized_n45/results/split_0/result_1_generalized_230106_frozen_metadata_for_MINES_split_00.csv_77226.91_.parquet"

# load in with polars and filter out only the predicted -> Lazydataframe
df_predicted = (
    pl.scan_parquet(path)
    #.filter(pl.col("Type") == "Predicted")
    )

In [ ]:
# check if any table from pickaxe has nulls!
path = "../../../MINE-Database/data/output/metacyc_generalized/20240601_lotus_generalized_n45/reactions_1_generalized_230106_frozen_metadata_for_MINES_split_*.parquet"

# load in with polars and filter out only the predicted -> Lazydataframe
df_predicted = (
    pl.scan_parquet(path)
    #.filter(pl.col("Type") == "Predicted")
    )


filtered_lazy_frame = (
    df_predicted
    .filter(
        pl.any_horizontal(pl.all().is_null())
    )
)

# Collect the filtered LazyFrame into a concrete DataFrame
filtered_df = filtered_lazy_frame.collect()

print("Filtered DataFrame:")
print(filtered_df)

In [5]:

%%time

import polars as pl


# change the configsetting, to see the full tables
pl.Config.set_tbl_rows(100)
pl.Config(fmt_str_lengths=550)

path = "../../../MINE-Database/data/output/metacyc_generalized/20240601_lotus_generalized_n45/results/split_0/result_1_generalized_230106_frozen_metadata_for_MINES_split_00.csv_77226.91_.parquet"

# load in with polars and filter out only the predicted -> Lazydataframe
df_predicted = (
    pl.scan_parquet(path)
    #.filter(pl.col("Type") == "Predicted")
    )

CPU times: user 3.84 ms, sys: 3.32 ms, total: 7.16 ms
Wall time: 502 ms


In [6]:
%%time

df_predicted.collect().columns

CPU times: user 46.5 s, sys: 9.76 s, total: 56.3 s
Wall time: 6.7 s


['ID',
 'predicted_reaction_ID equation',
 'predicted_reaction_SMILES equation',
 'predicted_reaction_Rxn hash',
 'predicted_reaction_Reaction rules',
 'predicted_compounds_Formula',
 'predicted_compounds_InChIKey',
 'predicted_compounds_SMILES']

In [9]:
df=df_predicted.collect()

df

ID,predicted_reaction_ID equation,predicted_reaction_SMILES equation,predicted_reaction_Rxn hash,predicted_reaction_Reaction rules,predicted_compounds_Formula,predicted_compounds_InChIKey,predicted_compounds_SMILES
str,str,str,str,str,str,str,str
"""http://www.wikidata.org/entity/Q104937439""","""(1) http://www.wikidata.org/entity/Q104937439[c0].(1) WATER[c0]>>(1) pkc23202935[c0]""","""COC1CC(OC2C(C)OC3(COC4C(C)OC(OC(C)C5(O)CCC6C7CC=C8CC(O)CCC8(C)C7CCC65C)CC4OCO3)CC2OC)OC(C)C1OC1CC(OC)C(OC2CC(OC)C(OC3OC(C)C(O)C(OC)C3OC(C)=O)C(C)O2)C(C)O1.O>>CCC1(C)C2=CCC3C1CCC1(C)C3CCC1(O)C(C)OC1CC3OCOC4(COC3C(C)O1)CC(OC)C(OC(OC(C)O)CC(OC)C(OC1CC(OC)C(OC3CC(OC)C(OC5OC(C)C(O)C(OC)C5OC(C)=O)C(C)O3)C(C)O1)C(O)C2)C(C)O4""","""Re1f85cc3ea1601acaae045254551bc4a50afe9963b71d9c7e036d5400761095f""","""rule0822""","""C66H110O25""","""RCDRFWFHJSHSKE-UHFFFAOYSA-N""","""CCC1(C)C2=CCC3C1CCC1(C)C3CCC1(O)C(C)OC1CC3OCOC4(COC3C(C)O1)CC(OC)C(OC(OC(C)O)CC(OC)C(OC1CC(OC)C(OC3CC(OC)C(OC5OC(C)C(O)C(OC)C5OC(C)=O)C(C)O3)C(C)O1)C(O)C2)C(C)O4"""
"""http://www.wikidata.org/entity/Q104995762""","""(1) http://www.wikidata.org/entity/Q104995762[c0].(1) PRENYL_ACCEPTOR_CoF[c0]>>(1) pkc14597182[c0]""","""CC1C(=O)COC2C1CC(OC1OC(CO)C(O)C(O)C1O)C2(C)O.O=P(O)(O)OP(=O)(O)O>>CC1C(=O)COC(C(C)(O)C(C)OC2OC(CO)C(O)C(O)C2O)C1OP(=O)(O)OP(=O)(O)O""","""R10a95a52f3a8c4d19f52250f6d6a3aea8d2f99540b992aefe7d58face569b9d4""","""rule0415""","""C16H30O16P2""","""ZWSYBQUGSGRGSV-UHFFFAOYSA-N""","""CC1C(=O)COC(C(C)(O)C(C)OC2OC(CO)C(O)C(O)C2O)C1OP(=O)(O)OP(=O)(O)O"""
"""http://www.wikidata.org/entity/Q105168359""","""(2) http://www.wikidata.org/entity/Q105168359[c0]>>(1) pkc15073428[c0].(1) pkc19713273[c0]""","""C=C1CCC2C(C)(C)C(OC(C)=O)CCC2(C)C1COc1ccc2ccc(=O)oc2c1>>C=C1CCC2C(C)(C)C(OC=O)CCC2(C)C1COc1ccc2ccc(=O)oc2c1.C=C1CCC2C(C)(C)C(OC(=O)CC)CCC2(C)C1COc1ccc2ccc(=O)oc2c1""","""Rc5ea950463553192471ca036bcb10e63c432145f7a33499ee48a612c6afbc635""","""rule0126""","""C25H30O5""","""IRLMTTAJFYCISV-UHFFFAOYSA-N""","""C=C1CCC2C(C)(C)C(OC=O)CCC2(C)C1COc1ccc2ccc(=O)oc2c1"""
"""http://www.wikidata.org/entity/Q105168359""","""(2) http://www.wikidata.org/entity/Q105168359[c0]>>(1) pkc15073428[c0].(1) pkc19713273[c0]""","""C=C1CCC2C(C)(C)C(OC(C)=O)CCC2(C)C1COc1ccc2ccc(=O)oc2c1>>C=C1CCC2C(C)(C)C(OC=O)CCC2(C)C1COc1ccc2ccc(=O)oc2c1.C=C1CCC2C(C)(C)C(OC(=O)CC)CCC2(C)C1COc1ccc2ccc(=O)oc2c1""","""Rc5ea950463553192471ca036bcb10e63c432145f7a33499ee48a612c6afbc635""","""rule0126""","""C27H34O5""","""XRKDCEFOFJKHJK-UHFFFAOYSA-N""","""C=C1CCC2C(C)(C)C(OC(=O)CC)CCC2(C)C1COc1ccc2ccc(=O)oc2c1"""
"""http://www.wikidata.org/entity/Q105015564""","""(2) http://www.wikidata.org/entity/Q105015564[c0]>>(1) pkc11848515[c0]""","""CC(=O)OC1CC=C2C(=O)OC3CC(C)C(C)(CC(O)c4ccoc4)C1C23C>>CC(=O)OC1CC=C2C(=O)OC3CC(C)(C(C)(O)OC4CC=C5C(=O)OC6CC(C)C(C)(CC(O)c7ccoc7)C4C56C)C(C)(CC(O)c4ccoc4)C1C23C""","""Ra7eb27498874dbe99e20f5009602dbabf0841b3cb0217b7f9dbf36a7aa21247d""","""rule0051""","""C44H56O12""","""XSSYDZRUTXWCRC-UHFFFAOYSA-N""","""CC(=O)OC1CC=C2C(=O)OC3CC(C)(C(C)(O)OC4CC=C5C(=O)OC6CC(C)C(C)(CC(O)c7ccoc7)C4C56C)C(C)(CC(O)c4ccoc4)C1C23C"""
"""http://www.wikidata.org/entity/Q105120170""","""(1) http://www.wikidata.org/entity/Q105120170[c0].(1) WATER[c0]>>(1) pkc15450471[c0]""","""CC(C)=CCc1cc(C(=O)C=Cc2ccccc2)c(O)c(CC=C(C)C)c1O.O>>CC(C)=CCC1=C(O)C(CC=C(C)C)C(O)C(C(=O)C=Cc2ccccc2)=C1O""","""Rae1394404546fdd4a354c9f903c6b6244370e609a39ee01d07089631c225acfd""","""rule1085""","""C25H30O4""","""LYXZGKNHOHPNQX-UHFFFAOYSA-N""","""CC(C)=CCC1=C(O)C(CC=C(C)C)C(O)C(C(=O)C=Cc2ccccc2)=C1O"""
"""http://www.wikidata.org/entity/Q105027428""","""(1) http://www.wikidata.org/entity/Q105027428[c0].(1) WATER[c0]>>(1) pkc12889759[c0]""","""CC1CCC(C(C)C2C(O)CC3C4CCC5CC(O)CCC5(C)C4CCC32C)=NC1.O>>CC1CCC(C(C)C2C(C)CCC3C(CCO)C(CC2O)C2CC(O)CCC32C)=NC1""","""Rf016026331ff24301eecd1580cbdc2bae2f708589204ad839c75d9e81aedba4a""","""rule0822""","""C27H47NO3""","""SFVLNQXLUYXFSI-UHFFFAOYSA-N""","""CC1CCC(C(C)C2C(C)CCC3C(CCO)C(CC2O)C2CC(O)CCC32C)=NC1"""
"""http://www.wikidata.

ToDo:
1. entries in single csv
2. remove https://wikidata....
3. split reaction collon